# ДЗ 2  
## Ранжирование: TF-IDF, матрица Document-Term, косинусная близость

### TfidfVectorizer

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()


# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1


# показать матрицу
print('X:\n', X.toarray(), '\n')

 
# чтобы получить сгенерированный словарВ ь, из приведенной структуры CountVectorizer
# порядок совпадает с матрицей
print('get_feature_names:', vectorizer.get_feature_names(), '\n')  # ['слово1', 'слово2', 'слово3', 'слово4']
 
    
# чтобы узнать индекс токена в словаре
print('vocabulary_.get:', vectorizer.vocabulary_.get('слово1')) # вернет 0
print('vocabulary_.get:', vectorizer.vocabulary_.get('слово4'), '\n') # вернет 3

 
# теперь можно быстро подсчитать вектор для нового документа
print('transform:', vectorizer.transform(['слово1 слово4 слово4']).toarray())  # результат [[0.36673901, 0, 0, 0.93032387]]


X:
 [[0.61366674 0.49681612 0.61366674 0.        ]
 [0.         0.62922751 0.77722116 0.        ]
 [0.92693676 0.3752176  0.         0.        ]
 [0.         0.         0.         1.        ]] 

get_feature_names: ['слово1', 'слово2', 'слово3', 'слово4'] 

vocabulary_.get: 0
vocabulary_.get: 3 

transform: [[0.36673901 0.         0.         0.93032387]]


### __Задача__:    

Реализуйте поиск, где 
- в качестве метода векторизации документов корпуса - **TF-IDF**
- формат хранения индекса - **матрица Document-Term**
- метрика близости пар (запрос, документ) - **косинусная близость**


Что должно быть в реализации:
- функция индексации корпуса, на выходе которой посчитанная матрица Document-Term 
- функция индексации запроса, на выходе которой посчитанный вектор запроса
- функция с реализацией подсчета близости запроса и документов корпуса, на выходе которой вектор, i-й элемент которого обозначает близость запроса с i-м документом корпуса
- главная функция, объединяющая все это вместе; на входе - запрос, на выходе - отсортированные по убыванию имена документов коллекции

В качестве корпуса возьмите корпус Друзей из первого задания.

**На что направлена эта задача:** 
Реализация от начала до конца механики поиска с использованием простых компонентов.


In [8]:
import os
from pymystem3 import Mystem
from string import punctuation
punctuation += '...' + '—' + '…' + '«»'
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
stopword = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trekc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def get_paths():
    curr_dir = os.getcwd()
    sub_dir = os.path.join(curr_dir, 'friends-data')
    
    paths = []
    names = []
    for root, dirs, files in os.walk(curr_dir):
        for name in files:
            if name.endswith('.ru.txt'):
                paths.append(os.path.join(root, name))
                names.append(name)
    
    return paths, names

In [11]:
# # Старое
# #from tqdm import tqdm
# #def preproc(paths):
# #    mystem = Mystem()
#     corpus = []
#     for path in tqdm(paths):
#         with open(path, 'r', encoding='utf-8-sig') as f:
#             text = f.read()
#         text = re.sub('[0-9a-zA-Z]+', '', text)
#         text = re.sub('-', '', text)
#         text = [word.lower().strip().strip(punctuation) for word in text.split()]
#         print(' '.join([x for x in text]))
#         text = mystem.lemmatize(' '.join([x for x in text]))
#         print(text)
#         text = [word for word in text if word != '']
#         text = [word for word in text if word != r' ']
#         text = [word for word in text if word != r'  ']
#         text = ' '.join([x for x in text if x not in stopword])
#         corpus.append(text)
        
#     return corpus

# paths, names = get_paths()
# corpus = preproc(paths)

In [12]:
from tqdm import tqdm
def preproc(paths):
    m = Mystem()
    corpus = []
    for path in tqdm(paths):
        with open(path, 'r', encoding='utf-8-sig') as f:
            text = f.read()
        text = re.sub('[0-9a-zA-Z]+', '', text)
        #text = re.sub('–', '', text)
        text = [word.lower().strip().strip(punctuation) for word in text.split()]
        #text = [word for word in text if word != '–']
        text = ' '.join([x for x in text if x not in stopword])
        corpus.append(text)
    
    lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    txtpart = lol(corpus, 1000)
    res = []
    for txtp in txtpart:
        alltexts = ' '.join([txt + ' br ' for txt in txtp])

        words = m.lemmatize(alltexts)
        doc = []
        for txt in words:
            if txt != '\n' and txt.strip() != '':
                if txt == 'br':
                    res.append(' '.join(doc))
                    doc = []
                else:
                    doc.append(txt)
    
    return res

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [14]:
def vect_corpus(corpus):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    
    return vectorizer, X

In [15]:
def trans_query(query, vectorizer):
    mystem = Mystem()
    query = re.sub('[0-9a-zA-Z]+','',query)
    query = [word.lower().strip().strip(punctuation) for word in query.split()]
    query = mystem.lemmatize(' '.join([x for x in query]))
    query = [word for word in query if word != '']
    query = ' '.join([x for x in query if x not in stopwords.words('russian')])
    
    return vectorizer.transform([query]).toarray()

In [16]:
def cos_counter(query, vectorizer, v_corpus):
    query = trans_query(query, vectorizer)
    cos_sim = v_corpus.toarray().dot(query.transpose())
    
    return cos_sim

In [41]:
def doc_finder(cos_result):
    sorted_сos_result = np.argsort(cos_result, axis=0)
    reversed_scr = np.flip(sorted_сos_result)
    print(reversed_scr)

    result = []
    for ind in reversed_scr:
        result.append(names[int(ind)])
    
    return result

In [24]:
paths, names = get_paths()
corpus = preproc(paths)
vectorizer, v_corpus = vect_corpus(corpus)

100%|███████████████████████████████████████████████████████████████████████████████| 165/165 [00:00<00:00, 208.45it/s]


In [42]:
from pprint import pprint
cos_result = cos_counter('мама мыла раму', vectorizer, v_corpus)
result = doc_finder(cos_result)
pprint(result)

[[ 10]
 [ 68]
 [ 36]
 [ 87]
 [ 66]
 [ 92]
 [ 69]
 [ 12]
 [ 67]
 [ 42]
 [ 75]
 [ 45]
 [  8]
 [ 70]
 [ 40]
 [154]
 [  7]
 [114]
 [ 39]
 [ 78]
 [  0]
 [163]
 [ 26]
 [ 31]
 [ 29]
 [124]
 [119]
 [153]
 [ 49]
 [ 24]
 [ 34]
 [121]
 [ 99]
 [  6]
 [ 94]
 [ 90]
 [164]
 [ 53]
 [  1]
 [ 63]
 [ 96]
 [ 11]
 [145]
 [ 33]
 [ 15]
 [125]
 [ 47]
 [  3]
 [  5]
 [ 95]
 [ 91]
 [ 98]
 [155]
 [ 62]
 [151]
 [144]
 [143]
 [129]
 [  9]
 [158]
 [ 89]
 [150]
 [ 35]
 [ 88]
 [ 44]
 [ 22]
 [ 83]
 [ 71]
 [ 72]
 [ 86]
 [ 80]
 [108]
 [ 84]
 [ 59]
 [ 82]
 [ 20]
 [156]
 [ 97]
 [ 56]
 [ 61]
 [ 58]
 [ 85]
 [ 51]
 [120]
 [136]
 [ 81]
 [127]
 [ 54]
 [ 32]
 [109]
 [ 55]
 [ 37]
 [ 28]
 [123]
 [  2]
 [ 27]
 [128]
 [ 30]
 [115]
 [160]
 [152]
 [131]
 [113]
 [112]
 [111]
 [ 25]
 [161]
 [157]
 [ 23]
 [162]
 [159]
 [130]
 [118]
 [ 13]
 [ 14]
 [117]
 [ 16]
 [ 17]
 [ 18]
 [ 19]
 [ 38]
 [ 21]
 [116]
 [122]
 [  4]
 [135]
 [149]
 [139]
 [142]
 [141]
 [140]
 [101]
 [100]
 [ 73]
 [ 74]
 [ 76]
 [103]
 [ 77]
 [138]
 [ 79]
 [132]
 [137]
 [133]

In [20]:
while True:
    query = input('Введите запрос:')
    cos_result = cos_counter(query, vectorizer, v_corpus)
    result = doc_finder(cos_result)
    pprint(result)

Введите запрос:мама
['Friends - 1x11 - The One With Mrs. Bing.ru.txt',
 'Friends - 4x02 - The One With The Cat.ru.txt',
 'Friends - 4x01 - The One With The Jellyfish.ru.txt',
 'Friends - 4x02 - The One With The Cat.Tv.ru.txt',
 'Friends - 1x13 - The One With The Boobies.ru.txt',
 'Friends - 4x01 - The One With The Jellyfish.Tv.ru.txt',
 'Friends - 2x16 - The One Where Joey Moves Out.ru.txt',
 'Friends - 2x22 - The One With The Two Parties.ru.txt',
 'Friends - 3x01 - The One With The Princess Leia Fantasy.ru.txt',
 'Friends - 1x09 - The One Where Underdog Gets Away.ru.txt',
 "Friends - 4x03 - The One With The 'Cuffs.ru.txt",
 'Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt',
 'Friends - 7x14 - The One Where They All Turn Thirty.ru.txt',
 'Friends - 1x08 - The One Where Nana Dies Twice.ru.txt',
 "Friends - 4x11 - The One With Phoebe's Uterus.ru.txt",
 "Friends - 7x23 - The One With Chandler And Monica's Wedding (1).ru.txt",
 'Friends - 2x11 - The One With The Lesbian Weddi

Введите запрос:наркотики секс
['Friends - 5x24-25 - The One In Vegas (2).ru.txt',
 'Friends - 6x09 - The One Where Ross Got High.ru.txt',
 "Friends - 4x11 - The One With Phoebe's Uterus.ru.txt",
 'Friends - 5x01 - The One After Ross Says Rachel.ru.txt',
 "Friends - 4x13 - The One With Rachel's Crush.ru.txt",
 "Friends - 7x01 - The One With Monica's Thunder.ru.txt",
 "Friends - 6x05 - The One With Joey's Porsche.ru.txt",
 "Friends - 5x13 - The One With Joey's Bag.ru.txt",
 "Friends - 2x04 - The One With Phoebe's Husband.ru.txt",
 'Friends - 6x12 - The One With The Joke.ru.txt',
 "Friends - 5x17 - The One With Rachel's Inadvertent Kiss.ru.txt",
 "Friends - 6x13 - The One With Rachel's Sister (1).ru.txt",
 'Friends - 5x08 - The One With The Thanksgiving Flashbacks.ru.txt',
 'Friends - 2x05 - The One With Five Steaks And An Eggplant.ru.txt',
 'Friends - 5x14 - The One Where Everybody Finds Out.ru.txt',
 'Friends - 7x05 - The One With The Engagement Picture.ru.txt',
 'Friends - 2x18 - The O

KeyboardInterrupt: Interrupted by user

In [ ]:
def cos_counter(query, vectorizer, v_corpus):
    query = trans_query(query, vectorizer)
    q_matrix = v_corpus.toarray().dot(query.transpose())
    sorted_q_matrix = np.argsort(q_matrix, axis=0)
    
    return np.flip(sorted_q_matrix)

In [ ]:
def get_paths():
    curr_dir = os.getcwd()
    sub_dir = os.path.join(curr_dir, 'friends-data')
    
    paths = []
    for root, dirs, files in os.walk(curr_dir):
        for name in files:
            if name.endswith('.ru.txt'):
                paths.append(os.path.join(root, name))
    
    return paths

функция индексации запроса, на выходе которой посчитанный вектор запроса

In [ ]:
def trans_query(query):
    return vectorizer.transform([query]).toarray()

функция с реализацией подсчета близости запроса и документов корпуса, на выходе которой вектор, i-й элемент которого обозначает близость запроса с i-м документом корпуса

In [ ]:
def finder(query):
    query = vectorizer.transform([query]).toarray()
    q_matrix = X.toarray().dot(query.transpose())
    sorted_q_matrix = np.argsort(q_matrix, axis=0)
    
    return np.flip(sorted_q_matrix)

q = finder('Мама мыла раму')
q

In [ ]:
data = pd.DataFrame(v.todense(), columns = vectorizer.get_feature_names())
data

In [ ]:
# чтобы получить сгенерированный словарВ ь, из приведенной структуры CountVectorizer
# порядок совпадает с матрицей
print('get_feature_names:', vectorizer.get_feature_names(), '\n')  # ['слово1', 'слово2', 'слово3', 'слово4']
 
    
# чтобы узнать индекс токена в словаре
print('vocabulary_.get:', vectorizer.vocabulary_.get('слово1')) # вернет 0
print('vocabulary_.get:', vectorizer.vocabulary_.get('слово4'), '\n') # вернет 3

 
# теперь можно быстро подсчитать вектор для нового документа
print('transform:', vectorizer.transform(['слово1 слово4 слово4']).toarray())  # результат [[0.36673901, 0, 0, 0.93032387]]


In [ ]:
# ind df
def finder(query):
    query = vectorizer.transform([query]).toarray()
    q_matrix = X.toarray().dot(query.transpose())

    q_df = pd.DataFrame(q_matrix)
    q_df = q_df.sort_values(by=[0], ascending=False)
    
    return list(q_df.index)

q = finder('Мама мыла раму')
q

### __Задача__:    

Реализуйте поиск, где 
- в качестве метода векторизации документов корпуса - **TF-IDF**
- формат хранения индекса - **матрица Document-Term**
- метрика близости пар (запрос, документ) - **косинусная близость**


Что должно быть в реализации:
- функция индексации корпуса, на выходе которой посчитанная матрица Document-Term 
- функция индексации запроса, на выходе которой посчитанный вектор запроса
- функция с реализацией подсчета близости запроса и документов корпуса, на выходе которой вектор, i-й элемент которого обозначает близость запроса с i-м документом корпуса
- главная функция, объединяющая все это вместе; на входе - запрос, на выходе - отсортированные по убыванию имена документов коллекции

В качестве корпуса возьмите корпус Друзей из первого задания.

**На что направлена эта задача:** 
Реализация от начала до конца механики поиска с использованием простых компонентов.
